<a href="https://colab.research.google.com/github/saraalharthi357/multiclass_diabetes_analysis/blob/main/multiclass_diabetes_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>